# Functional Encryption - Classification and information leakage

This code is made to convert the private layers in a format compatible with the **Reading in the dark project** which implements Quadratic Functional Encryption for real.

- make sure the Pytorch model is loaded is the appropriate path
- Load it back, transform and pickle the private layers to be readable for the Reading in the dark project.

In [77]:
# Allow to load packages from parent
import sys, os
sys.path.insert(1, os.path.realpath(os.path.pardir))

In [78]:
import torch
from math import log2, ceil

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils

import learn
from learn import collateral


In [80]:
PRIVATE_OUTPUT_SIZE = 4
N_CHARS = 10
N_FONTS = 2
prec=(3, 7, 5)

HIDDEN_WIDTH = 40

To load the Pytorch model you need to re-specify its structure.

In [81]:
class CollateralNet(nn.Module):
    def __init__(self):
        super(CollateralNet, self).__init__()
        self.proj1 = nn.Linear(784, HIDDEN_WIDTH)
        self.diag1 = nn.Linear(HIDDEN_WIDTH, PRIVATE_OUTPUT_SIZE, bias=False)

        # --- FFN for characters
        self.lin1 = nn.Linear(PRIVATE_OUTPUT_SIZE, 32)
        self.lin2 = nn.Linear(32, N_CHARS)

        # --- Junction
        self.jct = nn.Linear(PRIVATE_OUTPUT_SIZE, 784)

        # --- CNN for families
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, N_FONTS)

    def quad(self, x):
        # --- Quadratic
        x = x.view(-1, 784)
        x = self.proj1(x)
        x = x * x
        x = self.diag1(x)
        return x

    def char_net(self, x):
        # --- FFN
        x = F.relu(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

    def font_net(self, x):
        # --- Junction
        x = self.jct(x)
        x = x.view(-1, 1, 28, 28)

        # --- CNN
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def forward_char(self, x):
        x = self.quad(x)
        x = self.char_net(x)
        return F.log_softmax(x, dim=1)

    def forward_font(self, x):
        x = self.quad(x)
        x = self.font_net(x)
        return F.log_softmax(x, dim=1)
    
    # We add the ability to freeze some layers to ensure that the collateral task does
    # not modify the quadratic net
    
    def get_params(self, net):
        """Select the params for a given part of the net"""
        if net == 'quad':
            layers = [self.proj1, self.diag1]
        elif net == 'char':
            layers = [self.lin1, self.lin2]
        elif net == 'font':
            layers = [self.jct, self.fc1, self.fc2, self.conv1, self.conv2]
        else:
            raise AttributeError(f'{net} type not recognized')
        params = [p for layer in layers for p in layer.parameters()]
        return params

    def freeze(self, net):
        """Freeze a part of the net"""
        net_params = self.get_params(net)
        for param in net_params:
            param.requires_grad = False

    def unfreeze(self):
        """Unfreeze the net"""
        for param in self.parameters():
            param.requires_grad = True

This path needs to be adapter depending of your project

In [82]:
CODE_PATH = "/Users/tryffel/code/"
path = CODE_PATH + 'reading-in-the-dark/mnist/objects/ml_models/quad_conv.pt'
model = CollateralNet()
results = {}

model.load_state_dict(torch.load(path))
model.eval()

CollateralNet(
  (proj1): Linear(in_features=784, out_features=40, bias=True)
  (diag1): Linear(in_features=40, out_features=4, bias=False)
  (lin1): Linear(in_features=4, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=10, bias=True)
  (jct): Linear(in_features=4, out_features=784, bias=True)
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
)

In [83]:
model.proj1.weight

Parameter containing:
tensor([[-2., -2.,  2.,  ...,  0.,  2.,  3.],
        [ 2.,  0., -4.,  ...,  1.,  0., -2.],
        [ 3.,  0.,  0.,  ...,  0.,  0., -3.],
        ...,
        [-1.,  3., -2.,  ..., -2., -2.,  0.],
        [ 4., -3., -3.,  ...,  4., -3.,  3.],
        [ 0.,  0., -3.,  ..., -1., -2., -3.]], requires_grad=True)

In [84]:
model.proj1.weight.t().shape

torch.Size([784, 40])

In [85]:
data_prec, proj_prec, diag_prec = prec 
proj = torch.cat((model.proj1.bias.reshape(1, HIDDEN_WIDTH) / 2**data_prec, model.proj1.weight.t()), 0)
proj = proj.long().tolist()
diag = model.diag1.weight.t().long().tolist()

Pickle the private layers

In [86]:
import pickle

In [87]:
assert len(proj) == (784 + 1)
assert len(proj[0]) == len(diag)
assert len(diag[0]) == PRIVATE_OUTPUT_SIZE
model = (proj,diag)

with open('/Users/tryffel/code/reading-in-the-dark/mnist/objects/ml_models/torch_cl_large.mlm', 'wb') as f:
    pickle.dump(model, f)